## Indian Start-up Funding Analysis


### Project Scenario
The Team is trying to venture into the Indian start-up ecosystem. As the data expert of the team, I was tasked to:

- Investigate the ecosystem and propose the best course of action.
  - Analyze funding received by start-ups in India from 2018 to 2021.
  - Separate data for each year of funding will be provided.
  - Find the start-ups’ details, the funding amounts received, and the investors' information.

### Business Understanding
The Indian startup ecosystem has experienced significant growth and investment over the past few years. As a data expert tasked with investigating this ecosystem, our goal is to analyze the funding received by startups in India from 2018 to 2021 and provide insights to guide investment decisions. By examining the details of startups, funding amounts, and investor information, we aim to understand the trends, opportunities, and challenges within the Indian startup landscape.

### Objective
To analyze funding trends and dynamics within the Indian start-up ecosystem from 2018 to 2021 and propose strategic recommendations for the team's venture.

### Hypothesis Testing
Null Hypothesis(Ho): There is no siginificate difference in the amount of funding between startups in Bangalore.
Alternative Hypothesis(Ha): There is a siginificate difference in the amount of funding between startups in Banaglore.

### Business Questions
####
1.What sectors have shown the highest growth in terms of funding received over the past four years?

2.What locations within India have emerged as the primary hubs for startup activity and investment, and what factors contribute to their prominence?

3.Are there any notable differences in funding patterns between early-stage startups and more established companies?

4.Which sectors recieve the lowest level of funding and which sectors recieve the highest levels of funding in India.

5.Which investors have more impact on startups over the years?

6.What are the key characteristics of startups that successfully secure funding, and how do they differ from those that struggle to attract investment?

In [ ]:
from dotenv import dotenv_values 
import pyodbc 
import numpy as np
import pandas as pd                      
import re     
import seaborn as sns
import matplotlib.pyplot as plt
import statistics as stat 
import geopandas as gpd
#import geoplot as gplt
import folium
from folium.plugins import MarkerCluster
from scipy import stats        
#from geopy.geocoders import Nominatim    
from scipy.stats import ttest_ind    
import warnings
warnings.filterwarnings('ignore')

print("🛬 Imported all packages.", "Warnings hidden. 👻")

### First Data Set
The first data was from a database management system, that is MIRCORSOFT SQL SERVER. Connection was made to the database using an Open Database Connectivity standard library, pyodbc. <br>
Two tables were read from the databases. That is, <br>
Table 1: dbo.LP1_startup_funding2020 <br>
Table 2: dbo.LP1_startup_funding2021


In [ ]:
# Load environment variables from .env file into a dictionary
environment_variables = dotenv_values('.env')

# Get the values for the credentials you set in the '.env' file
server = environment_variables.get("SERVER")
database = environment_variables.get("DATABASE")
username = environment_variables.get("USERNAME")
password = environment_variables.get("PASSWORD")

In [ ]:
# Create a connection string
connection_string = f"Driver={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}"

In [ ]:
# Use the connect method of the pyodbc library and pass in the connection string.
# This will connect to the server and might take a few seconds to be complete. 
# Check your internet connection if it takes more time than necessary
connection = pyodbc.connect(connection_string)

In [ ]:
# Now the sql query to get the data is what what you see below.
# Note that you will not have permissions to insert delete or update this database table.
query = "Select * from dbo.LP1_startup_funding2020"
table_1 = pd.read_sql(query, connection)


In [ ]:
query = "Select * from dbo.LP1_startup_funding2021"
table_2 = pd.read_sql(query, connection)

In [ ]:
table_1.info()

In [ ]:
table_2.info()

In [ ]:
table_1.head(3)

In [ ]:
table_1[table_1['column10'].notna()]

#### Drop column10 in table_1 since it has just two values that are not null and are also just repetition of values in Stage column 

In [ ]:
# Drop column10
table_1.drop('column10', axis=1, inplace=True) if 'column10' in table_1.columns else table_1
table_1.head(3)

In [ ]:
# Function to clean the column names

def stripper(string: str, strip: list) -> str:
    """Strips a strip list from a given string and returns the string"""
    for s in strip:
        string = string.replace(s, '')
        
    return string
    
def replacer(string: str, replace: list) -> str:
    """Replaces each character in replace list with underscore given a string and returns the string"""
    for r in replace:
        string = string.replace(r, '_')
                
    return string
    
def clean_column_names(df):
    strip   = ['(', ')', '$']
    replace = [' ', '/'] 
    df.columns = [replacer(stripper(col_name.lower(), strip), replace) for col_name in df.columns]
    return df


In [ ]:
# Clean the column names
table_1 = clean_column_names(table_1)
table_2 = clean_column_names(table_2)

In [ ]:
table_1.info()

In [ ]:
table_2.info()

## Create year column to identify each dataset

In [ ]:
# Table 1: dbo.LP1_startup_funding2020
table_1['year'] = 2020

# Table 2: dbo.LP1_startup_funding2021
table_2['year'] = 2021

### First Data Set



In [ ]:
# First Data set
first_dataset = pd.concat([table_1, table_2], ignore_index=True)

first_dataset.info()


In [ ]:
first_dataset['amount'].unique()

## Second Data Set

In [ ]:
# Load dataset
second_dataset = pd.read_csv('datasets/individual_csv/startup_funding2019.csv')

second_dataset = clean_column_names(second_dataset)

second_dataset.info()

In [ ]:
second_dataset['amount'].unique()

## Third Data Set

In [ ]:
# Load dataset
third_dataset = pd.read_csv('datasets/individual_csv/startup_funding2018.csv')

third_dataset = clean_column_names(third_dataset)

third_dataset.info()

In [ ]:
third_dataset['amount'].unique()

In [ ]:
# Fix location values, take every letter before the first comma as headquarter
third_dataset['location'] = [location.split(',')[0] for location in third_dataset['location']]


In [ ]:
# Rename round_series to stage and location to headquarter
third_dataset.rename(columns={
    'company_name': 'company_brand', 
    'industry': 'sector', 
    'round_series': 'stage', 
    'about_company': 'what_it_does', 
    'location': 'headquarter'
    },
    inplace=True
)

third_dataset.info()

## Create Year column to identify each dataset


In [ ]:
# Create year column

# The second data is a flat-file name startup_funding2019.csv
second_dataset['year'] = 2019

# The third part of the data flat-file named startup_funding2018.csv
third_dataset['year']  = 2018

# Create the final concatenated dataset

In [ ]:
# Final Data set
final_dataset = pd.concat([first_dataset, second_dataset, third_dataset], ignore_index=True)

final_dataset.info()

In [ ]:
final_dataset.head()

### Cleaning Columns

- Headquarter & Sector column
    - If column value contains '#REF!', clean it and shift the row starting from that column by 1 step and until the stage column
    - Sanitize sector column if after cleaning and shifting, but the sector value is also present among unique values of the headquarter column
    - Fixes index 1297, 1312, 2155

In [ ]:
# Function to remove '#REF!' in a series
def remove_ref(value):
    if isinstance(value, str):
        value = value.replace('#REF!', '').strip()
            
    return value

# Columns of Interest 
columns = ['headquarter', 'investor']
for column in columns:    
    # Identify rows where column value contains '#REF!
    mask = final_dataset[column].str.contains('#REF!')
    
    # Fill missing values in mask with False
    mask.fillna(False, inplace=True)
    
    # Update the column by applying the remove_ref function to the column
    final_dataset.loc[mask, column] = final_dataset.loc[mask, column].apply(remove_ref)
    
    # Shift values in selected rows excluding the last column 'year'
    final_dataset.loc[mask, column:'stage'] = final_dataset.loc[mask, column:'stage'].shift(1, axis=1)


# Sanitisizing the sector column after shifting
mask = final_dataset['sector'].apply(lambda x: x in final_dataset['headquarter'].unique())

# Update 'headquarter' value with 'sector' value
final_dataset.loc[mask, 'headquarter'] = final_dataset.loc[mask, 'sector']

# Set the 'sector' value to NaN
final_dataset.loc[mask, 'sector'] = np.nan          

#### Replace 'None' string values with NaN element-wise allowing for consistent representation of missing values

In [ ]:
# Function replace None with NaN
def replace_none(value):
    if isinstance(value, str) and (value.lower() == 'none' or value.lower() == 'nan'):
        value = np.nan
    
    return value

# Apply the function to all columns
final_dataset = final_dataset.applymap(replace_none) # element-wise

In [ ]:
final_dataset.info()

#### If a value in stage column is a website link, its correct value is NaN

**Website link in stage column is https://docs.google.com/spreadsheets/d/1x9ziNeaz6auNChIHnMI8U6kS7knTr3byy_YBGfQaoUA/edit#gid=1861303593**

In [ ]:
# Function to remove website link from stage column
def remove_website_link(value):
    # Regular expression pattern to match website URLs that begin with http:// or https:// with an optional www
    pattern = r'https?://(?:www\.)?\w+\.\w+(?:/\S*)?'
    
    # Check if the value is a string and matches the pattern
    if isinstance(value, str) and re.match(pattern, value):
        return np.nan
    else:
        return value

In [ ]:
# Remove website link values from the stage column
final_dataset['stage'] = final_dataset['stage'].apply(remove_website_link)

#### Exchange rates 

[Source: OFX](https://www.ofx.com/en-au/forex-news/historical-exchange-rates/yearly-average-rates/)
```bash
exchange_rates = {
    2018: 0.014649,
    2019: 0.014209,
    2020: 0.013501,
    2021: 0.013527
}

In [ ]:
# Function to clean amount values
def floater(string):
    try:
        string = float(string)
    except ValueError:
        string = np.nan
    
    return string
    
def clean_amount(row): 
    amount = row[0]    
    year   = row['year'] 
    
    # Source: https://www.ofx.com/en-au/forex-news/historical-exchange-rates/yearly-average-rates/
    exchange_rates = {
        2018: 0.014649,
        2019: 0.014209,
        2020: 0.013501,
        2021: 0.013527
    }
    
    exchange_rate = exchange_rates[year]   
    
    # Convert to string
    amount = str(amount)   
    
    if isinstance(amount, str):        
        # Set of elements to replace
        to_replace = {' ', ','}

        # Replace each element in the set with an empty string
        for r in to_replace:
            amount = amount.replace(r, '')        
                        
        if amount == '' or amount == '—': 
            amount = np.nan
        # If the amount is in INR (Indian Rupees), convert it to USD using the conversion rate of the year
        elif '₹' in amount:
            amount = amount.replace('₹', '')
            amount = floater(amount) * exchange_rate
        
        # If the amount is in USD, remove the '$' symbol and convert it to a float
        elif '$' in amount:
            amount = amount.replace('$', '')
            amount = floater(amount)
        else:
            amount = floater(amount)

    
    return amount


In [ ]:
final_dataset.isna().sum()

#### If the investor value is a number or contains $, the correct value for amount if missing is the investor value, the correct value for stage is the old amount value and the investor value becomes NaN or missing

In [ ]:
# Identify rows where 'investor' value is numeric using clean amount function
mask = final_dataset[['investor', 'year']].apply(lambda row: pd.notna(clean_amount(row)), axis=1)

# Update 'stage' column with the 'amount' value if stage is NaN
stage_mask = final_dataset['stage'].isna()
final_dataset.loc[mask & stage_mask, 'stage']    = final_dataset.loc[mask, 'amount']

# Update 'amount' column with 'investor' value
final_dataset.loc[mask, 'amount']                = final_dataset.loc[mask, 'investor']

# Set 'investor' to NaN
final_dataset.loc[mask, 'investor']              = np.nan
         


#### If the 'stage' value is a number or contains $, the column 'what_it does' becomes its old value concatenated with the value in the 'founder' column. The correct value for 'founder' is the 'investor' value and the correct value for 'investor' is the 'amount' value and correct 'amount' becomes the old 'stage' value while the correct value for 'stage' is NaN or missing. 


In [ ]:
# Identify rows where 'stage' value is numeric using clean amount function
mask = final_dataset[['stage', 'year']].apply(lambda row: pd.notna(clean_amount(row)), axis=1)

# Update the 'what_it_does' column to its concatenation with 'founder' value
old_what_it_does = final_dataset.loc[mask, 'what_it_does']
old_founder      = final_dataset.loc[mask, 'founders']

final_dataset.loc[mask, 'what_it_does']    = old_what_it_does.fillna('') + ' ' + old_founder.fillna('')

# Update 'founder' column using the old 'investor' value
final_dataset.loc[mask, 'founders']        = final_dataset.loc[mask, 'investor']

# Update 'investor' column using the old 'amount' value
final_dataset.loc[mask, 'investor']        = final_dataset.loc[mask, 'amount']

# Update 'amount' column using the old 'stage' value
final_dataset.loc[mask, 'amount']          = final_dataset.loc[mask, 'stage']

# Set 'stage' to NaN
final_dataset.loc[mask, 'stage']           = np.nan

#### Clean and convert amounts to USD and rename colume from amount to amount($)

In [ ]:
# Clean and convert amounts to USD considering the average exchange rate per year
final_dataset['amount'] = final_dataset[['amount','year']].apply(lambda row: clean_amount(row), axis=1)


In [ ]:
final_dataset.rename(columns={'amount': 'amount($)'}, inplace=True)

#### Cleaning the Headquarter Column

In [ ]:
# Fix headquarter values, take every word before the first comma as headqurter
def splitter(value):
    """
    Splits a string by comma and returns the first part.

    Args:
        value (str or None): The value to be split.

    Returns:
        str or None: The first part of the string before the first comma, or the original value if the input is not a string.
    """
    return value.split(',')[0] if isinstance(value, str) else value

final_dataset['headquarter'] = [splitter(hq) for hq in final_dataset['headquarter']]

### Cleaning the Sector Column

In [ ]:
def heal_column_from_chars(df: pd.DataFrame, column: str = 'sector', chars: list = [',', ' ', '&', 'and', '/']) -> pd.DataFrame:
    """
    Returns a dataframe with the sector column having the least redundant value for sector.
    
    Parameters:
        df (pd.DataFrame): The DataFrame to clean.
        column (str): The name of the column to clean. Defaults to 'sector'.
        chars (list): A list of characters or substrings to handle. Defaults to [',', ' ', '&', 'and', '/']. Always start with ',' for sector 
        
    Returns:
        pd.DataFrame: The DataFrame with the specified column cleaned.
    """
    
    def count_char(value, char):
        """
        Finds all occurrences of char in value and returns the count.
        
        Parameters:
            value (str): The string in which to search for occurrences of char.
            char (str): The character to count occurrences of.
        
        Returns:
            int: The count of occurrences of char in value.
        """
        # Use re.findall to find all occurrences of char in value and return the count
        return len(re.findall(re.escape(char), str(value)))
    
    def char_to_nochar_value(char_value, char, no_char_column):
        """
        Find the equivalent value of char_value in no_char_column.
        
        Parameters:
            char_value (str): The string to process.
            char (str): The character to split the char_value string.
            no_char_column (pd.Series): The column containing unique values to search for the equivalent value.
        
        Returns:
            str: The equivalent value found in no_char_column.
        """
        
        # Default no char value        
        no_char_value = char_value 
        
        # Split by char, if char='and', "Food and Beverage" = ["Food ", " Beverage"]
        char_value    = char_value.split(char) 
        
        # If char is comma ',' or if char_value is not amongst no_char_column
        no_char_value = char_value[0] if char == ',' else no_char_value
        
        other_value = ''
            
        def find_index(where, value):
            """
            Find the index where the given value matches the elements in the Series 'where'.
            
            Parameters:
                where (pd.Series): The Series containing strings to search for the value.
                value (str): The value to search for.
            
            Returns:
                np.ndarray: The array of indices where the value matches in the Series.
            """
            return np.where(where.str.lower().unique() == value.strip().lower())[0]
            
        for value in char_value: 
            other_value = other_value + ' ' + value
            # Find the index of value in unique no_char_column
            indexof_value = find_index(where=no_char_column, value=value)
            
            # Find the index of other value in unique no_char_column
            indexof_othervalue = find_index(where=no_char_column, value=value)
            
            if indexof_value.size == 1:
                # If a unique match is found, update no_char_value
                no_char_value = no_char_column.unique()[indexof_value[0]]              
                break
            elif indexof_othervalue.size == 1:  
                # If a unique match is found, update no_char_value
                no_char_value = no_char_column.unique()[indexof_othervalue[0]]              
                break
            else:
                pass
                
        return no_char_value
        
    for char in chars: 
                
        char_list = [count_char(value, char) for value in df[column]]
                
        char_threshold =  0

        mask_char      = [x > char_threshold for x in char_list]    # Mask Boolean

        mask_no_char   = [not x for x in mask_char]
        
        char_column    = df.loc[mask_char, column]

        no_char_column = df.loc[mask_no_char, column]                

        # Convert 'char' column to no 'char' column in the dataframe if there is a no 'char' equivalent    
        df.loc[mask_char, column] = char_column.apply(lambda x: char_to_nochar_value(x, char, no_char_column))
    
    return df  


In [ ]:
# Cleaning the sector column
final_dataset = heal_column_from_chars(final_dataset)

- Sector- handling missing values

In [ ]:
def fill_missing_sector(value):
    """
    Fill missing sector values based on the content of 'what_it_does' column.

    Parameters:
    value (str): The content of the 'what_it_does' column.

    Returns:
    str: The assigned sector based on the content of 'what_it_does'.
    """
    stopwords   = ['it', "a", "an", "the", "and", "but", "or"]  # Add other stopwords to exclude
    
    # Add more sectors and keywords
    sector_keywords = {
        'Technology': ['platform', 'platforms', 'platform.', 'applications', 'digital', 'digitizes'],
        'Waste management': ['waste'],
        'Skill development': ['skill, development'], 
        'Commerce': ['ecommerce'],
        'Cosmetics':['skincare'],
        'Rental': ['space'],
        'HR': ['workforce'],
        'Finance': ['financial'],
        'Automobile': ['tyre'],
        'EdTech': ['edutech']
                        
    }
        
    sector  = 'Others' # Default sector if no match is found
    
    sectors = final_dataset['sector']
    
    values  = [v for v in value.split(' ') if v.lower() not in stopwords]
    
    for v in values:
         # Find the index of the sector in the sectors column
        index_sector = np.where(sectors.str.lower().unique() == v.lower())[0] # Find the index of sector in unique sectors
        
        # If a match is found, assign the corresponding sector
        if index_sector.size == 1:        
            sector = sectors.unique()[index_sector[0]]              
            break 
        # If no match is found, search for the word, v in the sector keywords dictionary and assign the corresponding sector 
        else:
            sector = next((sector for sector, keywords in sector_keywords.items() if v.lower() in keywords), sector)
    
    return sector

mask = final_dataset['sector'].isna()

final_dataset.loc[mask, 'sector'] = final_dataset.loc[mask, 'what_it_does'].apply(fill_missing_sector)

#### Replace falsely unique values or actual duplicates in categorical and string columns with their first occurence in the final_dataset

In [ ]:
# Heuristic Function to replace actual duplicates with first occurrence
def replace_col_duplicates(column):
    """
        Replaces duplicates values (identified through some heuristics) in a column with their first occurrence 
        Although the first occurrence might not be the best representation but it allows for consistency in values
        
        Parameter: column
        Returns  : column with consistent representation of values
    
    """
    actual_strings = {}  # Dictionary to store the first occurrence of each modified string
    
    def replace_string(string):
        actual_string = re.sub(r'[^\w]', '', string).lower() if isinstance(string, str) else string   # Replace all special characters including whitespaces with '' 
        if actual_string in actual_strings:
            return actual_strings[actual_string]
        else:
            actual_strings[actual_string] = string
            return string
    return column.apply(replace_string)


In [ ]:
columns_of_interest = ['company_brand', 'headquarter', 'sector', 'founders', 'investor', 'stage']

In [ ]:
def count_unique_values(columns):
    for col in columns:
        # Print count of unique items in columns of interest
        print(f'{col}: {len(final_dataset[col].unique())}\n')

In [ ]:
# Count of unique values before running replace_col_duplicates function
count_unique_values(columns_of_interest)

In [ ]:
# Apply the replace actual duplicates function to the string or categorical columns
for col in columns_of_interest:
    final_dataset[col] = replace_col_duplicates(final_dataset[col])


In [ ]:
# Count of unique values after running replace_col_duplicates function
count_unique_values(columns_of_interest)

In [ ]:
# Check the number of missing values in the columns of final dataset
final_dataset.isna().sum()

#### Drop duplicated rows in final data set

In [ ]:
# Drop duplicated from final dataset
final_dataset.drop_duplicates(inplace=True)

In [ ]:
# Again, check the number of missing values in the columns of final dataset
final_dataset.isna().sum()

#### Change each column (founded and year) to appropriate Data types

In [ ]:
final_dataset.info()

In [ ]:
# Founded is year and datatype should be integer 32 handling missing values gracefuly
final_dataset['founded'] = final_dataset['founded'].astype('Int32') # Int32 instead of int32

In [ ]:
# Year does not need to be int64 but int32
final_dataset['year'] = final_dataset['year'].astype('int32')

In [ ]:
final_dataset.info()

#### Handle missing values in the columns of the final dataset except company_brand, what_it_does and year which have no missing values

In [ ]:
final_dataset.describe(include='all')

In [ ]:
final_dataset.info()

In [ ]:
final_dataset.isna().any()

### Founded column

In [ ]:
final_dataset['founded'].unique()

In [ ]:
# Impute missing values in 'founded' column with median by year
founded_median_by_year = final_dataset['founded']

### Headquarter column

### Sector column

### Founders column

### Investor column

### Amount column

### Stage column

In [ ]:
final_dataset['stage'].unique()

### Fill missing values and replace Undisclosed values in 'stage' column with 'Venture - Series Unknown

In [ ]:
# Fill missing values in 'stage' column with 'Venture - Series Unknown
final_dataset['stage'].fillna('Venture - Series Unknown', inplace=True)

# Replace Undisclosed values in 'stage' column with 'Venture - Series Unknown
final_dataset['stage'].replace(to_replace='Undisclosed', value='Venture - Series Unknown', inplace=True)


In [ ]:
final_dataset['stage'].unique()

In [ ]:
final_dataset.isna().sum()

## Handling Missing Values in the Founders Column
### The dataset had 545 missing values for the founders, so we decided to drop the column for founders as we will not need it for analysis

In [ ]:
final_dataset.drop('founders', axis=1, inplace=True) 

## Handling Missing Values in the Sector column
### The sector column had 18 missing values so we filled them with the value Unknown

In [ ]:
final_dataset['sector'].unique()

In [ ]:
# Filling the missing values in sector column with Unkownn
final_dataset['sector'].fillna("Unknown", inplace=True)


In [ ]:
len(re.sub(r'[^\,]', '', "ar345547686,,4,5,,"))

## Handling Missing values in the Investor Column
 

In [ ]:
# Calculate the mode for each sector in the 'investor' column
mode_per_sector = final_dataset.groupby('sector')['investor'].transform(lambda x: x.mode().iloc[0] if not x.mode().empty else "Unknown")


# Fill missing values in the 'investor' column with the calculated mode per sector
final_dataset['investor'].fillna(mode_per_sector, inplace=True)


In [ ]:
final_dataset.isna().sum()


## Filling null values in the column for Head Quarter

In [ ]:
# Set seed for reproducibility
np.random.seed(2024)

# Identify the rows with missing 'headquarter' values
mask = final_dataset['headquarter'].isna()

# Get non-missing values for final_dataset['headquarter']
non_missing_hq = final_dataset.loc[mask==False, 'headquarter'] 

hq_missing     = mask.sum()

# Randomly sample non-missing values to fill missing values of size hq_missing, 114
hq_random      = np.random.choice(non_missing_hq, size=hq_missing)

# Fill missing values with randomly sampled headquarter values
final_dataset.loc[mask, 'headquarter'] = hq_random


### Filling Null Values in Founded Column

In [ ]:
# Calculate the median founded grouped by year values
founded_median_per_year = final_dataset.groupby('year')['founded'].median()

# Calculate the overall median of median_per_year
overall_median_per_year = founded_median_per_year.median()

# Fill NaN values in founded_median_per_year with the overall median of median_per_year
founded_median_per_year.fillna(overall_median_per_year, inplace=True)

# Fill missing values in the 'founded' column with the calculated median per year
final_dataset['founded'].fillna(final_dataset['year'].map(founded_median_per_year), inplace=True)

### Filling Null Values in the Amount column

In [ ]:
amount_median = final_dataset['amount($)'].median()
final_dataset['amount($)'].fillna(amount_median, inplace=True)

In [ ]:
final_dataset.isna().sum()

# Save datasets as flat files

In [ ]:
# Dataset from Microsoft SQL database
# first_dataset.to_csv('DataSets/individual_csv/startup_funding2020-2021.csv')

In [ ]:
# Final merged dataset with cleaned column names and cleaned amount values
# final_dataset.to_csv('DataSets/final_csv/startup_funding2018-2021.csv')

In [ ]:
final_dataset.hist()

### Test

In [ ]:
startups_in_bangalore = final_dataset[final_dataset['headquarter']=='Bangalore']
startups_not_in_bangalore = final_dataset[final_dataset['headquarter']!='Bangalore']

In [ ]:
t,p =stats.ttest_ind(startups_in_bangalore['amount($)'],startups_not_in_bangalore['amount($)'])
print(f't-value:{t}')
print(f'p-value:{p}')

In [ ]:
# Set Significance level
alpha = 0.05

#Compare p-value to the significance level
if p < alpha:
    print("We reject the null hypothesis. Which is there is no siginificate difference in the amount of funding between startups in Bangalore.")
else:
   print("We failed to reject the null hypothesis. There is a siginificate difference in the amount of funding between startups in Bangalore.")

In [ ]:
final_dataset.describe(include="all")

In [ ]:
# final_dataset['headquarter'].unique()

## Exploratory Data Analysis (EDA)

In [ ]:
final_dataset.info()

In [ ]:
final_dataset['stage'].unique()

#### What sectors have shown the highest growth in terms of funding received over the past four years?

In [ ]:
year_sector_2018 = final_dataset[final_dataset['year']==2018]
year_sector_2019 = final_dataset[final_dataset['year']==2019]
year_sector_2020 = final_dataset[final_dataset['year']==2020]
year_sector_2021 = final_dataset[final_dataset['year']==2021]


#Group 2018 data by sector and get the sum of the amount
sector_year_funding_2018 = year_sector_2018.groupby('sector')['amount($)'].sum()/ 1e9
sector_year_funding_2018_reset = sector_year_funding_2018.reset_index().sort_values(by='amount($)', ascending=False).head(5)

#Group 2019 data by sector and get the sum of the amount
sector_year_funding_2019 = year_sector_2019.groupby('sector')['amount($)'].sum()/ 1e9
sector_year_funding_2019_reset = sector_year_funding_2019.reset_index().sort_values(by='amount($)', ascending=False).head(5)

#Group 2020 data by sector and get the sum of the amount
sector_year_funding_2020 = year_sector_2020.groupby('sector')['amount($)'].sum()/ 1e9
sector_year_funding_2020_reset = sector_year_funding_2020.reset_index().sort_values(by='amount($)', ascending=False).head(5)

#Group 2021 data by sector and get the sum of the amount
sector_year_funding_2021 = year_sector_2021.groupby('sector')['amount($)'].sum()/ 1e9
sector_year_funding_2021_reset = sector_year_funding_2021.reset_index().sort_values(by='amount($)', ascending=False).head(5)

sector_year_funding_2018_reset

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(16, 9))

# Function to annotate each bar with the amount
def annotate_bars(data, ax):
    for index, value in enumerate(data['amount($)']):
        ax.text(value, index, f'{value:.2f}B', ha='left', va='center', fontsize=10)

# Plotting for 2018
sns.barplot(x='amount($)', y='sector', data=sector_year_funding_2018_reset, palette="viridis", ax=axes[0, 0])
annotate_bars(sector_year_funding_2018_reset, axes[0, 0])
axes[0, 0].set_title('Top 5 Sectors with Highest Growth in Funding in 2018')
axes[0, 0].set_xlabel('Average Growth in Funding (In Billions)')
axes[0, 0].set_ylabel('Top 5 Sectors in 2018')

# Plotting for 2019
sns.barplot(x='amount($)', y='sector', data=sector_year_funding_2019_reset, palette="viridis", ax=axes[0, 1])
annotate_bars(sector_year_funding_2019_reset, axes[0, 1])
axes[0, 1].set_title('Top 5 Sectors with Highest Growth in Funding in 2019')
axes[0, 1].set_xlabel('Average Growth in Funding (In Billions)')
axes[0, 1].set_ylabel('Top 5 Sectors in 2019')

# Plotting for 2020
sns.barplot(x='amount($)', y='sector', data=sector_year_funding_2020_reset, palette="viridis", ax=axes[1, 0])
annotate_bars(sector_year_funding_2020_reset, axes[1, 0])
axes[1, 0].set_title('Top 5 Sectors with Highest Growth in Funding in 2020')
axes[1, 0].set_xlabel('Average Growth in Funding (In Billions)')
axes[1, 0].set_ylabel('Top 5 Sectors in 2020')

# Plotting for 2021
sns.barplot(x='amount($)', y='sector', data=sector_year_funding_2021_reset, palette="viridis", ax=axes[1, 1])
annotate_bars(sector_year_funding_2021_reset, axes[1, 1])
axes[1, 1].set_title('Top 5 Sectors with Highest Growth in Funding in 2021')
axes[1, 1].set_xlabel('Average Growth in Funding (In Billions)')
axes[1, 1].set_ylabel('Top 5 Sectors in 2021')

# Adjust layout
plt.tight_layout()

# Show the plots
plt.show()

In [ ]:
# #Convert the 'growth' values to billions
# sector_year_funding_2018_reset['growth_billions'] = sector_year_funding_2018_reset['amount($)'] / 1e9

# # Plotting using seaborn
# plt.figure(figsize=(10, 6))
# bar_plot = sns.barplot(x='growth_billions', y='sector', data=sector_year_funding_2018_reset, palette="viridis")

# plt.title('Top 5 Sectors with Highest Growth in Funding in 2018')
# plt.xlabel('Average Growth in Funding (In Billions)')
# plt.ylabel('Top 5 Sectors in 2018')
# plt.show()

In [ ]:
# #Convert the 'growth' values to billions
# sector_year_funding_2019_reset['growth_billions'] = sector_year_funding_2019_reset['amount($)'] / 1e9

# # Plotting using seaborn
# plt.figure(figsize=(10, 6))
# bar_plot = sns.barplot(x='growth_billions', y='sector', data=sector_year_funding_2019_reset, palette="viridis")


# plt.title('Top 5 Sectors with Highest Growth in Funding in 2019')
# plt.xlabel('Average Growth in Funding (In Billions)')
# plt.ylabel('Top 5 Sectors in 2019')
# plt.show()

In [ ]:
# #Convert the 'growth' values to billions
# sector_year_funding_2020_reset['growth_billions'] = sector_year_funding_2020_reset['amount($)'] / 1e9

# # Plotting using seaborn
# plt.figure(figsize=(10, 6))
# bar_plot = sns.barplot(x='growth_billions', y='sector', data=sector_year_funding_2020_reset, palette="viridis")


# plt.title('Top 5 Sectors with Highest Growth in Funding in 2020')
# plt.xlabel('Average Growth in Funding (In Billions)')
# plt.ylabel('Top 5 Sectors in 2020')
# plt.show()

In [ ]:
# #Convert the 'growth' values to billions
# sector_year_funding_2021_reset['growth_billions'] = sector_year_funding_2021_reset['amount($)'] / 1e9

# # Plotting using seaborn
# plt.figure(figsize=(10, 6))
# bar_plot = sns.barplot(x='growth_billions', y='sector', data=sector_year_funding_2021_reset, palette="viridis")


# plt.title('Top 5 Sectors with Highest Growth in Funding in 2021')
# plt.xlabel('Average Growth in Funding (In Billions)')
# plt.ylabel('Top 5 Sectors in 2021')
# plt.show()

### What locations within India have emerged as the primary hubs for startup activity and investment, and what factors contribute to their prominence?

In [ ]:
# # Grouping the dataset based on the headquarters
# startups_by_location = final_dataset.groupby('headquarter')['amount($)'].sum()

# #Reset index and get the top 10 locations
# top_10_locations = startups_by_location.reset_index().sort_values(by ='amount($)', ascending = False).head(10)

# # top_10_locations

# # #Convert the 'growth' values to billions
# top_10_locations['growth_billions'] = top_10_locations['amount($)'] / 1e9

# # Plotting using seaborn
# plt.figure(figsize=(10, 6))
# bar_plot = sns.barplot(x='growth_billions', y='headquarter', data=top_10_locations, palette="viridis")
# # Add the numbers near the bars
# for i, v in enumerate(top_10_locations['growth_billions']):
#     bar_plot.text(v + 3, i + .25, str(v), color='black', fontweight='light')

# plt.title('Top 10 locations with the Highest Startup activity and Investment')
# plt.xlabel('Average Investment in Funding (In Billions)')
# plt.ylabel('Top 10 locations')
# plt.show()

In [ ]:
# Grouping the dataset based on the headquarters
startups_by_location = final_dataset.groupby('headquarter')['amount($)'].sum()
# Reset the index before assigning column names
top_10_locations_by_investment = startups_by_location.reset_index().sort_values(by ='amount($)', ascending = False).head(10)
top_10_locations_by_investment['growth_billions'] = top_10_locations_by_investment['amount($)'] / 1e9

# Assign column names
top_10_locations_by_investment.columns = ['headquarter', 'amount($)', 'growth_billions']
# top_10_locations_by_investment

# Initialize the Nominatim geocoder
geolocator = Nominatim(user_agent="my_geocoder")

# Function to retrieve coordinates for a location
def get_coordinates(location):
    try:
        location_info = geolocator.geocode(location)
        if location_info:
            return location_info.latitude, location_info.longitude
        else:
            print(f"Warning: Coordinates not found for {location}. Skipping.")
            return None, None
    except Exception as e:
        print(f"Error retrieving coordinates for {location}: {str(e)}")
        return None, None

# Apply the function to get coordinates for each location
top_10_locations_by_investment[['Latitude', 'Longitude']] = top_10_locations_by_investment['headquarter'].apply(lambda x: pd.Series(get_coordinates(x)))
top_10_locations_by_investment

In [ ]:
# # Create a map centered around the first location
map_top_10_locations_by_investment= folium.Map(location=[top_10_locations_by_investment['Latitude'].iloc[0], top_10_locations_by_investment['Longitude'].iloc[0]], zoom_start=3)
marker_cluster = MarkerCluster().add_to(map_top_10_locations_by_investment)

# Add a marker for each location to the MarkerCluster
for idx, row in top_10_locations_by_investment.iterrows():
    radius = int(row['growth_billions'] / 1e9)
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=folium.Popup(('<strong><font color ="green">'+row['headquarter']+'</font></strong><br>'+
                            '<strong>Total Investment (Billions): </strong><font color ="blue">'+str(row['growth_billions'])+'</font><br>'), max_width=250),
    ).add_to(marker_cluster)
map_top_10_locations_by_investment

In [ ]:
# startups_by_location2 = final_dataset['headquarter'].value_counts()

# # Reset index and get the top 10 locations
# top_10_locations2 = startups_by_location2.head(10).sort_values(ascending=False).reset_index()
# top_10_locations2.columns = ['headquarter', 'count']


# # Plotting using seaborn
# plt.figure(figsize=(10, 6))
# ax = sns.barplot(x='count', y='headquarter', data=top_10_locations2, palette="viridis")

# # Add the numbers near the bars
# for i, v in enumerate(top_10_locations2['count']):
#     ax.text(v + 3, i + .25, str(v), color='black', fontweight='light')

# plt.title('Top 10 locations with the Highest Startups')
# plt.xlabel('Number of Startups')
# plt.ylabel('Top 10 locations')
# plt.show()


In [ ]:
startups_by_location2 = final_dataset['headquarter'].value_counts()
top_10_locations = startups_by_location2.head(10).sort_values(ascending=False).reset_index()
top_10_locations.columns = ['Location', 'Number of Startups']

# Initialize the Nominatim geocoder
geolocator = Nominatim(user_agent="my_geocoder")

# Function to retrieve coordinates for a location
def get_coordinates(location):
    try:
        location_info = geolocator.geocode(location)
        if location_info:
            return location_info.latitude, location_info.longitude
        else:
            print(f"Error retrieving coordinates for {location}: Location not found")
            return None, None
    except Exception as e:
        print(f"Error retrieving coordinates for {location}: {str(e)}")
        return None, None
# Apply the function to get coordinates for each location
top_10_locations[['Latitude', 'Longitude']] = top_10_locations['Location'].apply(lambda x: pd.Series(get_coordinates(x)))

In [ ]:
top_10_locations

In [ ]:
# Create a map centered around the first location
map_top_10_locations = folium.Map(location=[top_10_locations['Latitude'].iloc[0], top_10_locations['Longitude'].iloc[0]], zoom_start=3)

# Create a MarkerCluster to cluster the markers
marker_cluster = MarkerCluster().add_to(map_top_10_locations)

# Add a marker for each location to the MarkerCluster
for idx, row in top_10_locations.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=folium.Popup(('<strong><font color ="green">'+row['Location']+'</font></strong><br>'+
                            '<strong>Number of Startups: </strong><font color ="blue">'+str(row['Number of Startups'])+'</font><br>'), max_width=250),
    ).add_to(marker_cluster)

# Display the map
map_top_10_locations

In [ ]:
# # # Grouping the dataset based on the headquarters
# startups_by_location2 = final_dataset['headquarter'].value_counts()

# #Reset index and get the top 10 locations
# top_10_locations2 = startups_by_location2.head(10).sort_values(ascending = False).reset_index()


# # Plotting using seaborn
# plt.figure(figsize=(10, 6))
# ax = sns.barplot(x='count', y='headquarter', data=top_10_locations2, palette="viridis")

# plt.title('Top 10 locations with the Highest Startups')
# plt.xlabel('Number of Startups')
# plt.ylabel('Top 10 locations')
# plt.show()


### Are there any notable differences in funding patterns between early-stage startups and more established companies?

In [ ]:
#Getting the median year for the founded coulmn
overall_median_per_year  # The overall median year is 2016

final_dataset['stage_of_startup'] = np.where(final_dataset['founded'] >= overall_median_per_year, 'Early Stage', 'Established')


In [ ]:
# Display the count of each category in the 'stage' column
stage_distribution = final_dataset['stage_of_startup'].value_counts()

# Print or visualize the distribution
print(stage_distribution)


In [ ]:
# Set up the figure and axis
fig, ax = plt.subplots(figsize=(8, 6))

# Plot countplot for 'stage'
sns.countplot(x='stage_of_startup', data=final_dataset, ax=ax, palette="viridis")

# Display the count above each bar
for p in ax.patches:
    ax.annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 10), textcoords='offset points')

plt.title('Distribution of Companies by Stage')
plt.xlabel('Stage')
plt.ylabel('Count')
plt.show()

#### Descriptive

In [ ]:
early_stage_stats = final_dataset[final_dataset['stage_of_startup'] == 'Early Stage']['amount($)'].describe()
established_stats = final_dataset[final_dataset['stage_of_startup'] == 'Established']['amount($)'].describe()

print("Early Stage Funding Statistics:")
print(early_stage_stats)

print("\nEstablished Funding Statistics:")
print(established_stats)


####
**Mean Funding Amount:** Early-stage companies have a slightly lower mean funding amount ($83.88 million) compared to established companies ($136.97 million).

**Variability (Standard Deviation):** Both groups exhibit high variability in funding amounts, as indicated by the large standard deviations.

**Minimum and Maximum Funding:** Both groups have a wide range of funding amounts, with early-stage companies having a minimum of $720 and a maximum of $150 billion, while established companies range from $40,900 to $70 billion.

**Percentiles (Q1, Median, Q3):** Early-stage companies generally have lower funding amounts at each percentile compared to established companies.

In [ ]:
early_stage_startups = final_dataset.groupby('stage_of_startup')['amount($)'].sum().reset_index()
early_stage_startups['amount_in_billions'] = early_stage_startups['amount($)'] / 1e9

plt.figure(figsize=(10, 6))
ax = sns.barplot(x='stage_of_startup', y='amount_in_billions', data=early_stage_startups, ci=None, palette="viridis")

for p in ax.patches:
    ax.annotate(f'{p.get_height():.2f}B', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 10), textcoords='offset points')

plt.title('Comparison of Funding Amounts between Early Stage and Established Companies')
plt.xlabel('Stage')
plt.ylabel('Amount ($B)')
plt.show()

 

#### Hypothesis Testing

#### 
Null Hypothesis(H0): There is no significant difference in the average funding amounts between early-stage startups and established companies.

Alternative Hypothesis(H1): There is a significant difference in the average funding amounts between early-stage startups and established companies.

In [ ]:
early_stage_funding = final_dataset[final_dataset['stage_of_startup'] == 'Early Stage']['amount($)']
established_funding = final_dataset[final_dataset['stage_of_startup'] == 'Established']['amount($)']

t_stat, p_value = ttest_ind(early_stage_funding, established_funding, nan_policy='omit')


# Set Significance level
alpha = 0.05
 
#Compare p-value to the significance level
if p_value < alpha:
    print('We reject the Null hypothesis.There is no significant difference in the average funding amounts between early_stage startups and established companies')
else:
   print("We fail to reject the Null hypothesis")


# print(f"T-statistic: {t_stat}")
# print(f"P-value: {p_value}")

### Which sectors recieve the lowest level of funding and which sectors recieve the highest levels of funding in India

In [ ]:
# Group data by sector and get the sum of the amount
sector_year_funding = final_dataset.groupby('sector')['amount($)'].sum()
sector_year_funding_reset = sector_year_funding.reset_index().sort_values(by='amount($)', ascending=False).head(5)

# Convert the 'growth' values to billions
sector_year_funding_reset['growth_billions'] = sector_year_funding_reset['amount($)'] / 1e9

def annotate_bars(data, ax):
    for index, value in enumerate(data['growth_billions']):
        ax.text(value, index, f'{value:.2f}B', ha='left', va='center', fontsize=10)

# Plotting using seaborn
plt.figure(figsize=(10, 6))
ax = sns.barplot(x='growth_billions', y='sector', data=sector_year_funding_reset, palette="viridis")

plt.title('Top 5 Sectors with Highest Growth in Funding')
plt.xlabel('Average Growth in Funding (In Billions)')
plt.ylabel('Top 5 Sectors')

# Annotate the bars with values
annotate_bars(sector_year_funding_reset, ax)
plt.show()
sector_year_funding_reset

In [ ]:
#Group data by sector and get the sum of the amount
sector_year_funding = final_dataset.groupby('sector')['amount($)'].sum()
sector_year_funding_reset = sector_year_funding.reset_index().sort_values(by='amount($)', ascending=False).tail(5)

#Convert the 'growth' values to billions
# sector_year_funding_reset['growth_billions'] = sector_year_funding_reset['amount($)'] / 1e9

def annotate_bars(data, ax):
    for index, value in enumerate(data['amount($)']):
        ax.text(value, index, f'{value:.2f}', ha='left', va='center', fontsize=10)

# Plotting using seaborn
plt.figure(figsize=(10, 6))
ax = sns.barplot(x='amount($)', y='sector', data=sector_year_funding_reset, palette="viridis")

annotate_bars(sector_year_funding_reset,ax)
plt.title('Least 5 Sectors with Lowest Growth in Funding')
plt.xlabel('Average Growth in Funding (In Thousands)')
plt.ylabel('Least 5 Sectors')
plt.show()
sector_year_funding_reset

### Which investors have more impact on startups over the years?

In [ ]:
# Group by investor and sum the investment amounts
investor_impact = final_dataset.groupby('investor')['amount($)'].sum()

# Get the top 10 investors with the highest total investment amounts
top_10_investors = investor_impact.nlargest(10)

#Return top 10 investors
top_10_investors

# Reset index and rename the columns
top_10_investors_reset = top_10_investors.reset_index()
top_10_investors_reset.columns = ['Investor', 'amount']
top_10_investors_reset

In [ ]:
# Convert the 'amount' values to billions
top_10_investors_reset['amount_billions'] = top_10_investors_reset['amount'] / 1e9

# Sort the DataFrame by 'amount_billions' column in descending order
top_10_investors_reset = top_10_investors_reset.sort_values(by='amount_billions', ascending=False)

# Plotting using seaborn
plt.figure(figsize=(10, 6))
bar_plot = sns.barplot(x='amount_billions', y='Investor', data=top_10_investors_reset, palette="viridis")

# Add data labels
for index, row in top_10_investors_reset.iterrows():
    bar_plot.text(row['amount_billions'], index, f'{row["amount_billions"]:.2f}B', va='center')

plt.title('Top 10 Investors Impact on Startups Over the Years')
plt.xlabel('Total Investment Amount (Billions $)')
plt.ylabel('Investor')
plt.show()



Made with 💖 [Team Curium](https://github.com/MumoMutiso/Indian-startup-collab)
<span style="color: #aaaaaa;">2024</span>